In [1]:
# Imports
import os
import bz2
import numpy as np
from PIL import TiffImagePlugin
from IPython.core.display import display
from PIL import Image as Img
import sys
import math
from functools import partial
sys.path.append("../")
from algorithms.flow import flow
from data_structures.location_graph import LocationGraph
from utils.dummy_writer import DummyWriter
from utils.image_writer import ImageWriter

In [2]:
def load_image_data() -> TiffImagePlugin.TiffImageFile:
    path = os.path.dirname(os.path.abspath('')) + '/data/'
    data = bz2.decompress(open(path + 'dem.tif.bz2', 'rb').read())
    with open(path + 'dem2.tiff', 'wb') as handle:
        handle.write(data)
    return Img.open(path + 'dem2.tiff')

def load_and_crop_data(crop_size, offset):
    raw_height_array = np.asarray(load_image_data())
    return raw_height_array[offset:offset+crop_size, offset:offset+crop_size]

In [3]:
# Params
size = 1000
frequency = 2000
job_name = "small"

def report_settings():
    num_images = int(size**2/frequency)
    print(f"Num images: {num_images}")
    total_size = ((size/1000)**2)*num_images
    if total_size > 1000:
        print(f"Total data size: {total_size/1000.0:.2f} GB")
    else:
        print(f"Total data size: {total_size:.2f} MB")
report_settings()

Num images: 500
Total data size: 500.00 MB


In [4]:
%%time
height_map = load_and_crop_data(size,0).tolist()
print("Creating Graph")
graph = LocationGraph(height_map)

Creating Graph
CPU times: user 15.7 s, sys: 244 ms, total: 16 s
Wall time: 15.9 s


In [5]:
%%time
print("Running flow simulation")
nodes_with_flow = flow(graph, DummyWriter())

Running flow simulation
CPU times: user 3.76 s, sys: 20.5 ms, total: 3.78 s
Wall time: 3.78 s


In [26]:
def normalize_values(i, max_log_value):
    i=math.log(i)*256/max_log_value
    val = int(i % 256)
    colour_ranges = [
        (255,int(128-val/2),int(128-val/2)),
        (255-val,0,0),
#         (255-val,0,0),
    ]
    category = int((i % 256*len(colour_ranges))/256)
    return colour_ranges[category]

def get_colour_function(flows):
    max_log_value = max([math.log(i) for i in flows])
    print(f"Max flow: {max(flows)}")
    print(f"Max log(flow): {max_log_value}")
    return partial(normalize_values, max_log_value=max_log_value)

In [27]:
flows = [n.flow for n in nodes_with_flow.ascending()]
writer = ImageWriter(size, frequency, job_name, get_colour_function(flows))

Max flow: 928.0282462013395
Max log(flow): 6.833062170039943
Writer will save images to /home/andrew/personal/river-flow/data/small/


In [28]:
def write_images(graph, writer):
    for node in graph.descending():
        writer.update(node)

In [29]:
%%time
print("Writing images from flow simulation")
write_images(graph,writer)

Writing images from flow simulation
Processed 2% of 1000000 images.
Processed 4% of 1000000 images.
Processed 6% of 1000000 images.
Processed 8% of 1000000 images.
Processed 10% of 1000000 images.
Processed 12% of 1000000 images.
Processed 14% of 1000000 images.
Processed 16% of 1000000 images.
Processed 18% of 1000000 images.
Processed 20% of 1000000 images.
Processed 22% of 1000000 images.
Processed 24% of 1000000 images.
Processed 26% of 1000000 images.
Processed 28% of 1000000 images.
Processed 30% of 1000000 images.
Processed 32% of 1000000 images.
Processed 34% of 1000000 images.
Processed 36% of 1000000 images.
Processed 38% of 1000000 images.
Processed 40% of 1000000 images.
Processed 42% of 1000000 images.
Processed 44% of 1000000 images.
Processed 46% of 1000000 images.
Processed 48% of 1000000 images.
Processed 50% of 1000000 images.
Processed 52% of 1000000 images.
Processed 54% of 1000000 images.
Processed 56% of 1000000 images.
Processed 58% of 1000000 images.
Processed 6